# Process fastqs from geo into count files 
NOTE: you will have to the fastqs you wish to process from GEO/SRA and place them in `/data/fastqs/` for this script to function  
If you wish to start with processed data, you can skip directly to notebooks in dir `/x02_analysis`

In [1]:
import pandas as pd 
import re

from Bio import SeqIO 

import gzip
import os 
import re 
import sys
import time
import bz2


### read in sample info data from geo meta file 

In [2]:
meta_df = pd.read_excel('../data/meta/GSE139657_meta.xls', 
                       skiprows=21,nrows=170 )

In [3]:
meta_df.head()

,Sample name,title,source name,organism,characteristics: experiment,molecule,description,processed data file,processed data file .1,raw file,raw file.1,raw file.2,raw file.3,raw file.4,raw file.5,raw file.6,raw file.7,Unnamed: 17
0,Sample 1,CMV_plasmid_0_p-a_GEN00105092,DNA plasmid,AAV,capsid packaging screen,AAV genome DNA,sequencing of AAV genome to determine packaing...,aav_packaging_all.csv,CMV_plasmid_0_p-a_GEN00105092.csv,LIB030216_GEN00105092_S22_L001_R1.fastq.bz2,LIB030216_GEN00105092_S22_L004_R2.fastq.bz2,LIB030216_GEN00105092_S22_L004_R1.fastq.bz2,LIB030216_GEN00105092_S22_L002_R2.fastq.bz2,LIB030216_GEN00105092_S22_L002_R1.fastq.bz2,LIB030216_GEN00105092_S22_L003_R1.fastq.bz2,LIB030216_GEN00105092_S22_L003_R2.fastq.bz2,LIB030216_GEN00105092_S22_L001_R2.fastq.bz2,NaN
1,Sample 2,CMV_plasmid_0_p-b_GEN00113847,DNA plasmid,AAV,capsid packaging screen,AAV genome DNA,sequencing of AAV genome to determine packaing...,aav_packaging_all.csv,CMV_plasmid_0_p-b_GEN00113847.csv,LIB031744_GEN00113847_S101_L004_R1.fastq.bz2,LIB031744_GEN00113847_S101_L003_R1.fastq.bz2,LIB031744_GEN00113847_S101_L003_R2.fastq.bz2,LIB031744_GEN00113847_S101_L004_R2.fastq.bz2,LIB031744_GEN00113847_S101_L002_R1.fastq.bz2,LIB031744_GEN00113847_S101_L002_R2.fastq.bz2,LIB031744_GEN00113847_S101_L001_R2.fastq.bz2,LIB031744_GEN00113847_S101_L001_R1.fastq.bz2,NaN
2,Sample 3,CMV_plasmid_0_p-c_GEN00113848,DNA plasmid,AAV,capsid packaging screen,AAV genome DNA,sequencing of AAV genome to determine packaing...,aav_packaging_all.csv,CMV_plasmid_0_p-c_GEN00113848.csv,LIB031744_GEN00113848_S102_L004_R1.fastq.bz2,LIB031744_GEN00113848_S102_L001_R2.fastq.bz2,LIB031744_GEN00113848_S102_L003_R2.fastq.bz2,LIB031744_GEN00113848_S102_L004_R2.fastq.bz2,LIB031744_GEN00113848_S102_L002_R1.fastq.bz2,LIB031744_GEN00113848_S102_L001_R1.fastq.bz2,LIB031744_GEN00113848_S102_L002_R2.fastq.bz2,LIB031744_GEN00113848_S102_L003_R1.fastq.bz2,NaN
3,Sample 4,CMV_virus_1_a_GEN00105095,AAV Capsid,AAV,capsid packaging screen,AAV genome DNA,sequencing of AAV genome to determine packaing...,aav_packaging_all.csv,CMV_virus_1_a_GEN00105095.csv,LIB030216_GEN00105095_S25_L001_R2.fastq.bz2,LIB030216_GEN00105095_S25_L003_R2.fastq.bz2,LIB030216_GEN00105095_S25_L003_R1.fastq.bz2,LIB030216_GEN00105095_S25_L002_R2.fastq.bz2,LIB030216_GEN00105095_S25_L004_R1.fastq.bz2,LIB030216_GEN00105095_S25_L001_R1.fastq.bz2,LIB030216_GEN00105095_S25_L004_R2.fastq.bz2,LIB030216_GEN00105095_S25_L002_R1.fastq.bz2,NaN
4,Sample 5,CMV_virus_1_a_GEN00105099,AAV Capsid,AAV,capsid packaging screen,AAV genome DNA,sequencing of AAV genome to determine packaing...,aav_packaging_all.csv,CMV_virus_1_a_GEN00105099.csv,LIB030216_GEN00105099_S29_L003_R1.fastq.bz2,LIB030216_GEN00105099_S29_L002_R1.fastq.bz2,LIB030216_GEN00105099_S29_L001_R2.fastq.bz2,LIB030216_GEN00105099_S29_L004_R2.fastq.bz2,LIB030216_GEN00105099_S29_L001_R1.fastq.bz2,LIB030216_GEN00105099_S29_L002_R2.fastq.bz2,LIB030216_GEN00105099_S29_L003_R2.fastq.bz2,LIB030216_GEN00105099_S29_L004_R1.fastq.bz2,NaN


functions used to count barcodes and write counts 

In [4]:
def tabulate(key, counts):
    # add 1 to count for this key, or add it to the dictionary
    if key in counts:
        counts[key] += 1
    else:
        counts[key] = 1

def write_counts(dict, outputfile):
    # write sequences and counts to a text file
#     print outputfile
    file_out = open(outputfile, 'w')
    file_out.write('barcode,count\n')
    for w in sorted(dict, key=dict.get, reverse=True):
        file_out.write('{seq},{num}\n'.format(seq=w, num=str(dict[w])))
    file_out.close() 
    

### using a row from the geo meta data, compute the counts for each barcode across all associated fastqs

In [5]:
def count_ligated_bars(output_path = '../data/',
                       count_dir_name = 'counts',
                       log_out_name = 'count_log_file',
                       meta_df_row = None, 
                      limit=None):
    
    lib_file = os.path.join(output_path,'fastq',meta_df_row['raw file'])
    save_name = meta_df_row['title']
#     print (gen_id)
    output_counts_dir = os.path.join(output_path, count_dir_name)
    out_file_full = os.path.join(output_counts_dir , '%s.txt' % log_out_name)
#     if from_top:
#         if os.path.isfile(out_file_full):
#             os.remove(out_file_full)
    if os.path.isfile(out_file_full):
        log = open(out_file_full, 'a')
    else:
        log = open(out_file_full, 'w')
        log.write('reads\tbarcodes\tbarcodes_per_read\tlib_name\n')
    lig_dict ={}
    bar_count = 0
    read_count = 0 
#     print 'counting lib %s' % lib_file
    lig_dict = {}
 
    full_file_list = []
    for lane in ['L001', 'L002', 'L003','L004']:
        for read_direction in ['R1','R2']:
            lane_sub = re.sub('L001', lane, lib_file)
            full_file_name = re.sub('R1', read_direction,lane_sub )
            print (full_file_name)
            if os.path.isfile(full_file_name) == False:
                print ("file %s does not exsits" % full_file_name)
                continue
            full_file_list.append(full_file_name)
            if 'gz' in full_file_name:
                handle = gzip.open(full_file_name)
            if 'bz' in full_file_name:
                print ("bz file")
                handle = bz2.open(full_file_name, 'rt')

            for idx, read in enumerate(SeqIO.parse(handle,'fastq')):
                read_count += 1 
                if limit:
                    if idx > limit : break 
                if 'R1' in read_direction:
                    seq = read.seq
                else:
                    seq = read.seq.reverse_complement()
    #             print seq # to debug
                middle_expr = '(?<=CCAC)([ACTG]{20})(?=CCAC)'
                matches = re.findall(middle_expr, str(seq))
                if matches:
                    for match in matches:
                        bar_count += 1
                        tabulate(match, lig_dict)

    print ("writing counts for %s" % save_name)
    assert len(full_file_list) == 8 , full_file_list
    log.write('%s\t%s\t%s\t%s\n' % (read_count,bar_count, (bar_count/float(read_count)), save_name))
    write_counts(lig_dict, os.path.join(output_counts_dir, '%s.csv' % (save_name)))
    log.close()

example counting the barcodes for one fo the sample

In [6]:
count_ligated_bars(meta_df_row = meta_df.iloc[2])

../data/fastq/LIB031744_GEN00113848_S102_L004_R1.fastq.bz2
bz file
../data/fastq/LIB031744_GEN00113848_S102_L004_R2.fastq.bz2
bz file
../data/fastq/LIB031744_GEN00113848_S102_L004_R1.fastq.bz2
bz file
../data/fastq/LIB031744_GEN00113848_S102_L004_R2.fastq.bz2
bz file
../data/fastq/LIB031744_GEN00113848_S102_L004_R1.fastq.bz2
bz file
../data/fastq/LIB031744_GEN00113848_S102_L004_R2.fastq.bz2
bz file
../data/fastq/LIB031744_GEN00113848_S102_L004_R1.fastq.bz2
bz file
../data/fastq/LIB031744_GEN00113848_S102_L004_R2.fastq.bz2
bz file
writing counts for CMV_plasmid_0_p-c_GEN00113848


### merge the barcode counts onto the meta-data table

In [11]:
barcode_to_mutant_table = pd.read_csv('../data/meta/AAV2scan_chip_lookup_table.txt')
def assemble_dataframe(meta_df_row=meta_df.iloc[2], mutant_info_df = barcode_to_mutant_table ,save=False):
    filename = meta_df_row['title']
    count_file = os.path.join('../data','counts','%s.csv' % filename )
    count_df = pd.read_csv(count_file)
    count_with_mutant_info_df = barcode_to_mutant_table.merge(count_df, on='barcode',how='left')
    if save:
        count_with_mutant_info_df.to_csv(os.path.join('../data/counts', "%s_dataframe.csv" % filename))
    return count_with_mutant_info_df

example for counts above

In [12]:
count_data_with_mutant_info_df = assemble_dataframe()
count_data_with_mutant_info_df.head()

,aa,abs_pos,barcode,codon,enzyme,is_wt_aa,is_wt_codon,lib_type,tile_num,wt_bc,aa-codon,count
0,*,1.0,CACTGTCACACACTGACACT,TAA,bbsi,0,0,sub,0.0,0,*-TAA,632.0
1,*,1.0,CTGTGAGTGTGAGAGACACT,TAA,bbsi,0,0,sub,0.0,0,*-TAA,616.0
2,*,1.0,CTCTCACACAGTGAGTCTGA,TAG,bbsi,0,0,sub,0.0,0,*-TAG,108.0
3,*,1.0,CAGAGACAGAGTCTGTCACT,TAG,bbsi,0,0,sub,0.0,0,*-TAG,136.0
4,*,1.0,ACACTGTCTCTGTCAGACAG,TGA,bbsi,0,0,sub,0.0,0,*-TGA,1048.0
